In [ ]:
pip install scikit-optimize category_encoders

Barrier


In [ ]:
import pandas as pd # pandas for data manipulation and dataframes
import os # os to work with local machines operating system
import kagglehub # kagglehub lets us download the dataset from the kaggle website

path = kagglehub.dataset_download("pilarpieiro/tabular-dataset-ready-for-malicious-url-detection")
print("Path to dataset files:", path)
csv_path = os.path.join(path, "test_dataset.csv")
df = pd.read_csv(csv_path)
print(list(df.columns))

In [ ]:
df['source'].value_counts() # displ

In [ ]:
from sklearn.model_selection import train_test_split

X = df.drop(columns='label')
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify=y, random_state =3)

In [ ]:
from sklearn.pipeline import Pipeline
from category_encoders.target_encoder import TargetEncoder
from xgboost import XGBClassifier

estimators = [
    ('encoder', TargetEncoder()),
    ('clf', XGBClassifier(random_state=3))
]
pipe = Pipeline(steps=estimators)
pipe

In [ ]:
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

search_space = {
   'clf__max_depth' : Integer(2,8),
   'clf__learning_rate' : Real(0.001, 1.0, prior='log-uniform'),
   'clf__subsample' : Real(0.5,1.0),
   'clf__colsample_bytree' : Real(0.5,1.0),
   'clf__colsample_bylevel' : Real(0.5,1.0),
   'clf__colsample_bynode' : Real(0.5,1.0),
   'clf__reg_alpha' : Real(0.0,10.0),
   'clf__reg_lambda' : Real(0.0,10.0),
   'clf__gamma' : Real(0.0,10.0),
}

opt = BayesSearchCV(pipe, search_space, cv=3, n_iter=10, scoring='roc_auc', random_state=3)

In [ ]:
opt.fit(X_train, y_train)

In [ ]:
opt.best_estimator_

In [ ]:
opt.best_score_

In [ ]:
opt.score(X_test, y_test)

In [ ]:
opt.predict(X_test)

In [ ]:
opt.predict_proba(X_test)

In [ ]:
opt.best_estimator_.steps

In [ ]:
from xgboost import plot_importance

xgboost_step = opt.best_estimator_.steps[1]
xgboost_model = xgboost_step[1]
plot_importance(xgboost_model)